<a href="https://colab.research.google.com/github/lracefetrj/mineracaodados/blob/main/Embedding_Bertimbau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
resp = pd.read_csv('/content/drive/MyDrive/amostra_recurso.csv')

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.9 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads

tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', do_lower_case=False)
model = AutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased')

In [ ]:
import torch
import torch.nn as nn

#**Cria embedding a partir de texto**


In [ ]:
def create_embedding(text):
  input_ids = tokenizer.encode(text, return_tensors='pt')

  with torch.no_grad():
      outs = model(input_ids)
      encoded = outs[0][0, 1:-1]
      mean_embedding = torch.mean(encoded, dim=0)
      #print("Tensor medio")
      print(mean_embedding)
      #print(encoded.ndim)
      #print(mean_embedding.shape)
      return mean_embedding



#**Lê recurso**

In [ ]:
def read_text(dataframe, linha, coluna):
    texto = dataframe.at[linha, coluna]
    return texto

#**Identifica fim do paragrafo**

In [ ]:
import re

def detect_paragraph_end(text):
    # Expressão regular para identificar pontos que indicam o fim de um parágrafo
    regex = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s'

    # Encontre todas as correspondências na string
    matches = re.finditer(regex, text)

    # Extraia os índices de início e fim de cada correspondência
    paragraph_ends = [(match.start(), match.end()) for match in matches]

    return paragraph_ends

# Texto de exemplo
text = "Isso é um parágrafo. Por exemplo, Sr. João está feliz. 1.5 é um número. Um parágrafo final."

# Detecte os pontos que indicam o fim de um parágrafo
paragraph_end_indices = detect_paragraph_end(text)

# Imprima os índices onde os pontos indicam o fim de um parágrafo
for start, end in paragraph_end_indices:
    print(f"Parágrafo termina entre os índices {start} e {end}: '{text[start:end]}'")

conteudo = read_text(resp,1,'recurso')
input_ids = tokenizer.encode(conteudo, return_tensors='pt')
#Limitacao de tamanho do texto para usar o modelo , para teste foi limitada a dimensao do tensor em 512
input_ids512 = input_ids[:, :512]

Parágrafo termina entre os índices 20 e 21: ' '
Parágrafo termina entre os índices 54 e 55: ' '
Parágrafo termina entre os índices 71 e 72: ' '


#**Extrai fragmentos do texto**
Necessário devido à limitação de tamanho imposta pelo modelo

In [ ]:
def extract_fragments(text, max_words_per_fragment):
    words = text.split()  # Divide o texto em palavras
    fragments = []
    current_fragment = []

    for word in words:
        if len(current_fragment) + len(word.split()) <= max_words_per_fragment:
            current_fragment.append(word)
        else:
            fragments.append(" ".join(current_fragment))
            current_fragment = [word]

    # Adiciona o último fragmento, se houver
    if current_fragment:
        fragments.append(" ".join(current_fragment))

    return fragments


## **Início do processamento**

In [ ]:
# Texto
text = read_text(resp,1,'recurso')

tensores_fragmentos = []

# Quantidade máxima de palavras por fragmento
max_words_per_fragment = 170

# Extrai os fragmentos
fragments = extract_fragments(text, max_words_per_fragment)

# Imprime os fragmentos
for idx, fragment in enumerate(fragments, start=1):
    print(f"Fragmento {idx}: {fragment}")

#Cria embeddings
for idx, fragment in enumerate(fragments, start=1):
    print(f"Criando embedding do fragmento {idx} ... ")
    tensor_fragmento = create_embedding(fragment)
    tensores_fragmentos.append(tensor_fragmento)

print("Tensores fragmentos")
print(tensores_fragmentos)
tensor_final = torch.stack(tensores_fragmentos)

print("Tensor final")
print(tensor_final)
tensor_final_media = torch.mean(tensor_final, dim=0)
#Imprime Embedding do texto completo
print("Embedding do texto completo a partir da média")
print(tensor_final_media)
print(tensor_final_media.shape)

Fragmento 1: ADVOCACIA-GERAL DA UNIÃO PROCURADORIA-GERAL FEDERAL PROCURADORIA REGIONAL FEDERAL 2a REGIÃO NPREV - SUBNÚCLEO FINALÍSTICO ATUAÇÃO TÉCNICA ESPECIALIZADA PREVIDENCIÁRIA TRIBUNAIS - EATE/TRF2/PREV AV. NILO PEÇANHA, 151 - CENTRO, RIO DE JANEIRO/RJ, CEP 20020-100 EXCELENTÍSSIMO(A) SENHOR(A) DESEMBARGADOR(A) DO(A) GABINETE 03 NÚMERO: 0216608-09.2017.4.02.5107 RECORRENTE(S): INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS RECORRIDO(S): GESSE JAMES ELOY SANTOS INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS, pessoa jurídica de direito público, representado(a) pelo membro da Advocacia-Geral da União infra assinado(a), vem, respeitosamente, à presença de Vossa Excelência, interpor, com fulcro no art. 105, inciso III, alínea 'a' da Constituição Federal da República Federativa do Brasil, vem interpor RECURSO ESPECIAL pelas razões que seguem, em anexo, requerendo o seu recebimento, processamento e remessa ao Colendo Superior Tribunal de Justiça, na forma da lei. RAZÓES DO RECURSO ESPECIAL DA TEM

In [ ]:
}